In [1]:
import pandas as pd
import cv2
import os
import numpy as np
import faiss

Resize and normalize training

In [2]:
import warnings
warnings.simplefilter("ignore")

In [3]:
training_df = pd.DataFrame()

training_path = "archive\Training Data"

for folder in os.listdir(training_path):
    folder_path = os.path.join(training_path, folder)
    for img in os.listdir(folder_path)[:200]:
        img_path = os.path.join(folder_path, img)
        temp_df = pd.DataFrame({  # Usar lista para que sea una entrada válida
        "Path": [img_path],
        "Class": [folder]
        })
    
        # Usar pd.concat para agregar el DataFrame temporal a training_df
        training_df = pd.concat([training_df, temp_df], ignore_index=True)

### HISTOGRAMAS 

In [4]:
paths_db = pd.read_csv("database\db.csv", index_col=0)
paths = paths_db["path"].values.tolist()

In [5]:
histograms = []
dimension = 256
for path in paths:
    img = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    histograms.append(cv2.calcHist([img], [0], None, [dimension], [0, 256]).squeeze())

HISTOGRAMAS (Aplicamos np.array y squeeze para cambiar el formato de [[0], [0], ...]) a [0, 0, ...]

In [6]:
index = faiss.IndexFlatL2(dimension)

vectores = np.array(histograms)

faiss.normalize_L2(vectores)

index.add(vectores)

# Guardar el índice en la carpeta database
faiss.write_index(index, "database\histograms.index")

Por si acaso

In [7]:
test_img_path = "archive\Testing Data\Coast\Coast-Test (1).jpeg"
img_test = cv2.imread(test_img_path)
img_test_gray = cv2.cvtColor(img_test, cv2.COLOR_BGR2GRAY)

test_hist = np.array(cv2.calcHist(img_test_gray, 
                                    channels=[0], 
                                    mask=None, 
                                    histSize=[256], 
                                    ranges=[0, 256])).squeeze().reshape(1, 256)

print(test_hist.shape)

distances, indices = index.search(test_hist, k=5)  # search for 5 nearest neighbors

print("Indices of nearest neighbors:", indices)
print("Distances to nearest neighbors:", distances)

(1, 256)
Indices of nearest neighbors: [[395 705 601  80 712]]
Distances to nearest neighbors: [[1555.7057 1555.8376 1557.54   1558.0826 1558.4724]]
